# TECH CHALLENGE
Tech Challenge é o projeto que engloba os conhecimentos obtidos em todas as disciplinas presentes na fase. Esta é uma atividade que, em princípio, deve ser desenvolvida em grupo. É importante atentar-se ao prazo de entrega, uma vez que essa atividade é obrigatória, valendo 90% da nota de todas as disciplinas da fase.

**O PROBLEMA**

No Tech Challenge desta fase, você precisa executar o fine-tuning de um foundation model (Llama, BERT, MISTRAL etc.), utilizando o dataset "The AmazonTitles-1.3MM". O modelo treinado deverá:


*   Receber perguntas com um contexto obtido por meio do arquivo json “trn.json”
*   A partir do prompt formado pela pergunta do usuário sobre o título do produto, o modelo deverá gerar uma resposta baseada na pergunta do usuário trazendo como resultado do aprendizado do fine-tuning os dados da sua descrição que está contido dentro do dataset.

### Fluxo de trabalho atualizado:
1. Escolha do Dataset:
Descrição: o The AmazonTitles-1.3MM consiste em consultas textuais reais de usuários e títulos associados de produtos relevantes encontrados na Amazon e suas descrições, medidos por ações implícitas ou explícitas dos usuários.

2. Preparação do Dataset:
Faça o download do dataset AmazonTitles-1.3MM e utilize o arquivo “trn.json”. Nele, você utilizará as colunas “title” e “content”, que contêm título e descrição respectivamente. Prepare os prompts para o fine-tuning garantindo que estejam organizados de maneira adequada para o treinamento do modelo escolhido. Limpe e pré-processe os dados conforme necessário para o modelo escolhido.

3. Chamada do Foundation Model
Importe o foundation model que será utilizado e faça um teste apresentando o resultado atual do modelo antes do treinamento (para que se obtenha uma base de análise após o fine-tuning), e então será possível avaliar a diferença do resultado gerado.

4. Execução do Fine-Tuning:
Execute o fine-tuning do foundation model selecionado (por exemplo,
BERT, GPT, Llama) utilizando o dataset preparado. Documente o processo de fine-tuning, incluindo os parâmetros utilizados e qualquer ajuste específico realizado no modelo.

5. Geração de Respostas:
Configure o modelo treinado para receber perguntas dos usuários. O modelo deverá gerar uma resposta baseada na pergunta do usuário e nos dados provenientes do fine-tuning, incluindo as fontes fornecidas. O que esperamos para o entregável?



*   Documento detalhando o processo de seleção e preparação do dataset.
*   Descrição do processo de fine-tuning do modelo, com detalhes dos parâmetros e ajustes utilizados. Código-fonte do processo de fine- tuning.
*   Um vídeo demonstrando o modelo treinado gerando respostas a partir de perguntas do usuário e utilizando o contexto obtido por meio treinamento com o fine-tuning.

In [1]:
# Montando volume compartilhado no google drive para salvar os arquivos localmente
# Isso nos permite reuso em diversos pontos desse notebook
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Instalação de Pacotes

In [2]:
# Instalando pacotes necessários.
# Decidimos utilizar o unsloth para acelerar o processo de fine-tuning, já que o dataset é muito grande
!pip install bert-tensorflow
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers==0.0.27 "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 5.9 MB/s eta 0:00:00
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8lwdbmj9/unsloth_1a6054295bfc4f7a85a66ed72328cc4d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8lwdbmj9/unsloth_1a6054295bfc4f7a85a66ed72328cc4d
  Resolved https://github.com/unslothai/unsloth.git to commit 539adef0f8aaaed43aaafab5aa952aa933f70c0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.4 MB/s eta 0:00:00


# Preparação do Dataset

Nos códigos abaixos:



*   Fazemos o download do dataset a partir do google drive
*   Extraímos o arquivo trn.json, que contém todos os dados que nós queremos
*   Processamos e limpamos o dataset, removendo campos em branco, preparando para o fine-tuning.



### Download do Dataset

In [3]:
!gdown 12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=40ded8c9-2c88-4797-880d-2b8e05e32df0
To: /content/LF-Amazon-1.3M.raw.zip
100% 890M/890M [00:08<00:00, 105MB/s]


### Extração do Dataset

In [4]:
import zipfile
drive = '/content/drive/MyDrive/FIAP/TC_03'
local = '/content/datasets'

zip_path = '/content/LF-Amazon-1.3M.raw.zip'
extract_to = f'{local}'
zipfile.ZipFile(zip_path, 'r').extractall(extract_to)

import gzip
import shutil

# Lê o arquivo JSON compactado em pedaços de 100.000 linhas, armazena os dados em um DataFrame.
datasets_path = f'{local}/LF-Amazon-1.3M'
trn_gz_filepath = f'{datasets_path}/trn.json.gz'

trn_file_path = '/content/datasets/trn.json'  # Local onde o arquivo extraído será salvo

# Abrir o arquivo .gz e extrair o conteúdo
with gzip.open(trn_gz_filepath, 'rb') as f_in:
    with open(trn_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

### Leitura e Pré-processamento do Dataset

Prepara conjunto de dados estruturado e formatado para o fine-tuning,  organizando inputs e outputs de maneira que o modelo aprenda a responder corretamente a instruções específicas, utilizando exemplos práticos de entradas (títulos) e saídas (descrições) de produtos. Essa estrutura é fundamental para treinar o modelo a gerar saídas coerentes e relevantes ao contexto.

In [5]:
# Vamos ler o arquivo json e salvar os dados já na estrutura necessária para o fine-tuning do modelo
# A estrutura segue o seguinte padrão:
# {
#    "instruction": [],
#    "input": [],
#    "output": []
# }


import gc
import json

instructions = []
inputs = []
outputs = []

with open(trn_file_path, 'r', encoding='utf-8') as file:
  for line in file:
    json_data = json.loads(line.strip())

    # Verificar se 'title' e 'content' não estão vazios
    if all(json_data.get(attr) for attr in ['title', 'content']):
        instructions.append("Me dê detalhes sobre o seguinte produto")
        inputs.append(json_data["title"])
        outputs.append(json_data["content"])

processed_data = {
    "instruction": instructions,
    "input": inputs,
    "output": outputs
}

# Salvando o dataset processado e limpo em um arquivo para reuso futuro
dataset_file = r'/content/drive/MyDrive/FIAP/TC_03/datasets/processed.json'
with open(dataset_file, 'w', encoding='utf-8') as file:
    json.dump(processed_data, file, ensure_ascii=False, indent=4)

# Limpa memória
del processed_data, instructions, inputs, outputs
gc.collect()

122

## Carregamento e Configuração do Modelo

Carrega um modelo pré-treinado usando a biblioteca unsloth e configura esse modelo para uso em inferência, carregando-o de forma otimizada para maximizar o desempenho e economizar memória.

In [6]:
# Modelos suportados pelo Unsloth
# Lista para referência

# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ]

import torch

from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Decidimos utilizar o modelo Llama
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Configuração do Modelo com PEFT

Parameter-Efficient Fine-Tuning é necessário para otimizar o processo de fine tuning da LLM (LLaMA-3). PEFT permite que apenas uma pequena fração dos parâmetros do modelo seja ajustada durante o treinamento.



In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #16,
    target_modules = ["q_proj", "k proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.9.post3 patched 32 layers with 0 QKV layers, 32 O layers and 32 MLP layers.


## Preparação do Dataset e Formatação dos Prompts

Define o formato de prompt que será usado para treinar o modelo e uma função para formatar os dados de entrada no formato definido.

In [8]:
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }


dataset_path = '/content/drive/MyDrive/FIAP/TC_03/datasets/processed.json'

dataset = load_dataset("json", data_files=dataset_path, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

formatted_dataset_path = '/content/drive/MyDrive/FIAP/TC_03/datasets/model_formatted_prompt.json'

with open(formatted_dataset_path, 'w') as f:
    json.dump(dataset.to_dict(), f, indent=4)

# Separar em test e train
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# Removendo variáveis temporárias
del dataset, split_dataset
gc.collect()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

0

## Carregamento do Dataset e Treinamento

Carrega o dataset preparado, mapeia os prompts e treina o modelo usando *SFTTrainer*.

In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    dataset_num_proc=1,
    args = TrainingArguments(
      per_device_train_batch_size = 2, # Número de amostras por dispositivo (GPU/CPU) em cada lote durante o treinamento.
      gradient_accumulation_steps = 4, # Número de passos de gradiente a serem acumulados antes de uma atualização do modelo.
      warmup_steps = 5, # Número de passos iniciais para aquecer o otimizador antes de iniciar a taxa de aprendizado padrão.
      max_steps = 60, # Número total de passos de treinamento a serem executados.
      learning_rate = 2e-4, # Taxa de aprendizado usada pelo otimizador para ajustar os pesos do modelo.
      fp16 = not is_bfloat16_supported(), # Usa precisão de ponto flutuante de 16 bits se bfloat16 não for suportado.
      bf16 = is_bfloat16_supported(), # Usa precisão bfloat16 se suportado pelo hardware.
      logging_steps = 1, # Número de passos entre cada registro de métricas de treinamento no log.
      optim = "adamw_8bit", # Otimizador usado durante o treinamento, configurado para usar 8 bits para otimização de memória.
      weight_decay = 0.01, # Fator de decaimento aplicado aos pesos para evitar sobreajuste durante o treinamento.
      lr_scheduler_type = "linear", # Tipo de agendamento da taxa de aprendizado, configurado para ajuste linear.
      seed = 3407, # Semente usada para garantir reprodutibilidade no treinamento.
      output_dir = "outputs", # Diretório onde os resultados e checkpoints do treinamento serão salvos.
      # fp16_full_eval = True, # Avaliação feita com precisão de ponto flutuante de 16 bits.
      # per_device_eval_batch_size = 2, # Número de amostras por dispositivo (GPU/CPU) em cada lote durante a avaliação.
      # eval_accumulation_steps = 4, # Número de passos de avaliação a serem acumulados antes de uma atualização.
      # evaluation_strategy = "steps", # Estratégia de avaliação configurada para ser feita a cada passo especificado.
      # eval_steps = 1, # Intervalo de passos entre cada avaliação do modelo durante o treinamento.
    ),
)

Map:   0%|          | 0/1251362 [00:00<?, ? examples/s]

Map:   0%|          | 0/139041 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer_stats = trainer.train()

# Salva modelo em disco para reuso
model.save_pretrained("/content/drive/MyDrive/FIAP/TC_03/model/model-unsloth/Meta-Llama-3.1-8B")
# Salva tokenizer em disco para reuso
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP/TC_03/model/model-unsloth/Meta-Llama-3.1-8B")

# Limpa memória
del trainer, trainer_stats, model, tokenizer, train_dataset, test_dataset
gc.collect()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,251,362 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 39,321,600


Step,Training Loss
1,2.973400
2,3.033100
3,2.922800
4,2.661700
5,2.816400
6,2.416200
7,2.369900
8,1.963600
9,2.048100
10,1.683100


129

In [11]:
# Limpa memória da GPU
with torch.no_grad():
    torch.cuda.empty_cache()

# eval_results = trainer.evaluate()
# print(f"Acurácia do modelo: {eval_results.get('eval_accuracy', 'Não disponível'):.2f}")

## Inferência com o Modelo Treinado

Realiza a inferência com o modelo treinado, gerando uma descrição do produto de entrada fornecido e exibindo a saída gerada.

In [17]:
import gc
import torch
from transformers import TextStreamer

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/FIAP/TC_03/model/model-unsloth/Meta-Llama-3.1-8B",
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)

inputs = tokenizer([
    prompt.format(
        "Me dê detalhes sobre o seguinte produto",
        "Girls Ballet Tutu Neon Blue",
        "",
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
# temperature - nível de alucinação do modelo
# outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)
outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128, temperature=0.1)
#tokenizer.batch_decode(outputs)

# Limpa a memória para permitir execuções consecutivas
del model, tokenizer, inputs, outputs, text_streamer
gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Me dê detalhes sobre o seguinte produto

### Input:
Girls Ballet Tutu Neon Blue

### Response:
Girls Ballet Tutu Neon Blue<|end_of_text|>
